## Predict climbing strength:  "The V grade you can send 90-100% of routes" (Any information)

In [ ]:
import pandas as pd
Data = pd.read_csv("formatted_data.csv")

y = Data['The_V_grade_you_can_send_90-100%_of_routes_']
X = Data.drop(columns=['The_V_grade_you_can_send_90-100%_of_routes_', 'Hardest_V_Grade_ever_climbed_', 
                       'Hardest_V_Grade_climbed_in_the_Last_3_months']) # remove target and correlated features
print(y.shape, X.shape)

KeyError: 'The_V_grade_you_can_send_90-100%_of_routes_'

In [ ]:
# Scale data